# Interpretable Machine Learning
## Exercise Sheet: 2

### This exercise sheet covers chapters 5.1+5.2 from the IML book by Christoph Molnar
Kristin Blesch (blesch@leibniz-bips.de)<br>
Niklas Koenen (koenen@leibniz-bips.de)
<hr style="border:1.5px solid gray"> </hr>

# 1) Linear Regression
The linear model is one of the first and best-studied models for regression
problems. However, it makes some (strong) assumptions about the given data. 
In the following exercise, you will learn how to fit and interpret a linear 
model and investigate its limitations. 

Let's consider a linear regression model 
for three features $p=3$ of the form
$$
\hat{f}(x) = \beta_0 + x_1 \beta_1 + x_2 \beta_2 + x_3\beta_3 = X \beta
$$
and the data is given in the csv-file `sheet_2_lin_data.csv`.

**a)** Load the data as a `pandas` data frame and visualize in a 2D plot how each 
feature ($X_1, X_2, X_3$) affects the output $Y$. Preprocess the loaded data frame
with `pandas.melt` and use the function `seaborn.scatterplot` with specified 
grouping argument (`hue`) for the visualization.

**Solution:**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb
sb.set_theme() # Because seaborn theme looks awesome
import pandas as pd

# Load the data from csv-file as a pandas data frame
linear_data = pd.read_csv("sheet_2_lin_data.csv")
# Preprocess the data frame so that we have only the columns "Y", "X" and "feature"
dat = pd.melt(linear_data, 
              id_vars = ['Y'], 
              value_vars = ["X1", "X2", "X3"],
              var_name = "feature",
              value_name = "X")
# Plot the data
plt.figure(figsize=(15, 10))
fig = sb.scatterplot(data = dat, x = "X", y = "Y", hue = "feature")

**b)** Determine the _design matrix_ $X$ of the given data as a `numpy` array (don't 
forget the column of ones for the intercept $\beta_0$). Using this matrix $X$, 
estimate the regression coefficients $\beta$ of the linear model by the method of least 
squares and calculate the corresponding standard error of the estimates.  
**Hint:** The estimated linear regression coefficients are given by 
$\hat{\beta} = (X^T X)^{-1} X^T Y$ and the standard error
is given by the square root of the diagonal entries of $(X^TX)^{-1} \hat{\sigma}^2$ with 
$\hat{\sigma}^2 = \frac{1}{n - p} \sum_{i=1}^n (y_i- \hat{y}_i)^2$.

**Solution:**

In [ ]:
import numpy as np

# Determine the design matrix X
X = linear_data.copy()
X.insert(1, "X0", 1)
X = X.loc[:,"X0":"X3"].to_numpy()

# Calculate the estimated regression coefficients (X * X^T)^-1
X_XT_inv = np.linalg.inv(np.matmul(X.T, X))

# Outcome Y as a numpy array
Y = linear_data["Y"].to_numpy()

# beta_hat = (X * X^T)^-1 * X^T * Y (see hint)
beta_hat = np.matmul(X_XT_inv, np.matmul(X.T, Y))

# Calculate the standard error estimate variance (see hint)
sigma_2 = np.sum((np.matmul(X, beta_hat) - Y)**2) / (X.shape[0] - 3)
standard_error = np.diag(sigma_2 * X_XT_inv)**0.5

# Print the result
pd.DataFrame(data = {"beta_hat": beta_hat, "SE": standard_error}, index = ["b0", "b1", "b2", "b3"])

**c)** Calculate the $R^2$-value. What does the $R^2$-value mean in this context?

**Solution:**

In [ ]:
# Calculate the squared sum of the error terms
sse = np.sum((np.matmul(X, beta_hat) - Y)**2)

# Calculate the squared sum of the data variance
sst = np.sum((Y - np.mean(Y))**2)

# Calculate and print the R²-value
"R²-value: {:.4f}".format(1 - sse / sst)

The R²-value takes values between 0 and 1 and describes how much variance in the true outcome is explained by the model and how well the model describes the given data. Since in this case the R²-value is almost 1, it is a very good fit and if we look only at the prediction performance, the linear model is a perfect choice for the given data.

**d)** What happens to the outcome if we increase the first feature value $X_1$ by three? Which feature
is the most important one?

**Solution:**

In [ ]:
# The outcome Y increases by
beta_hat[1] * 3

In [ ]:
# Calculate feature importance
importance = np.abs(beta_hat / standard_error)
print(importance)

# Get the most important feature
most_important_feature = np.argmax(importance)

# Print the most important feature
"X" + str(most_important_feature)

**e)** The data was sampled by the data generating process 
$$f(x) = x_1 + 2 x_2 + 3 x_3 + \varepsilon.$$ 
Does the most important feature from part d) make sense regarding the data 
generating process? Explain your answer!

**Solution:**

At a first look: No! If you look at the data generating process, $x_3$ would have to be the most important feature because it gets the highest weighting with $3$ i.e. it has the biggest influence on output $y$. Apart from the intercept, $x_1$ would have to be the least important feature because it has the smallest weight. Thus, it seems that not all the requirements for a linear model are satisfied.

For example, the input features $x_1$ and $x_3$ are highly correlated, which screws up the estimation of the coefficients:

In [ ]:
# Plot the correlation
sb.heatmap(linear_data.corr(), annot = True, cmap = "Greys")
plt.show()

<hr style="border:1.5px solid gray"> </hr>

# 2) Logistic Regression

## a) Introduction of odds ratio
**i)** Suppose we have an ordinary and uniformly distributed six-sided
dice described by a random variable $Y_1$. What are the odds of rolling a 
$5$ or a $6$?

**Solution:**

The odds is defined by the probability of an event divided by
the probability of its counter event, i.e. $\text{odds} = \frac{\mathbb{P}(Y_1 \geq 5)}{\mathbb{P}(Y_1 < 5)}$. Hence
$$
\text{odds}_{Y_1} = \frac{\mathbb{P}(Y_1 \geq 5)}{\mathbb{P}(Y_1 < 5)} = \frac{\mathbb{P}(Y_1 \geq 5)}{1 - \mathbb{P}(Y_1 \geq 5)} = \frac{\frac{2}{6}}{1 - \frac{2}{6}} = \frac{1}{2} \hat{=}\ \text{"1-to-2"}.
$$

**ii)** Let's consider a loaded dice $Y_2$ that rolls a $6$ with a probability of $\frac{1}{3}$
and a $5$ with a probability of $\frac{1}{12}$. What are the odds for the event 
$Y_2 \geq 5$ now?

**Solution:**

We get
$$
\text{odds}_{Y_2} = \frac{\mathbb{P}(Y_2 \geq 5)}{\mathbb{P}(Y_2 < 5)} = \frac{\frac{1}{3} + \frac{1}{12}}{1 - \left(\frac{1}{3} + \frac{1}{12}\right)} = \frac{\frac{5}{12}}{1 - \frac{5}{12}} = \frac{5}{7}\hat{=}\ \text{"5-to-7"}.
$$

**iii)** How have the odds changed from dice $Y_1$ to $Y_2$, i.e. by how much is it
higher/lower? This factor is called the _odds ratio_.

**Solution:**

$$
\text{odds ratio} = \frac{\text{odds}_{Y_2}}{\text{odds}_{Y_1}} = \frac{\frac{5}{7}}{\frac{1}{2}} = \frac{10}{7} \approx 1.4286.
$$
This means that the odds of rolling a number greater than $5$ have increased by
$1.4285$ times if the second dice is thrown instead of the first one.

## b) Logistic Regression in Python
In this exercise, a logistic model is fitted on the [iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set) indicating whether 
a given plant of the iris genus is a member of the species 'virginica' or not. For 
this purpose, we use the mighty package [`scikit-learn`](https://scikit-learn.org/stable/).

**i)** Load the iris dataset from `sklearn.datasets` and save the features with 
corresponding output in one `pandas` dataframe. Then adjust the output to the 
binary classification problem described above.

**Solution:**

In [ ]:
from sklearn.datasets import load_iris
import pandas as pd

# Load the iris dataset
iris_dataset = load_iris()

# I changed the feature names a bit (not necessary)
feature_names = [i.rsplit(' ', maxsplit = 1)[0] for i in iris_dataset.feature_names]

# Move the iris dataset to pandas
iris_df = pd.DataFrame(iris_dataset.data, columns = feature_names)

# Adjust the output, remember  0 = 'setosa', 1 = 'versicolour' and 2 = 'virginica'
iris_df['virginica'] = iris_dataset.target == 2
iris_df

**ii)** Visualize this dataframe with `seaborn.pairplot` with grouping argument `virginica`.
What do you think which feature incremented by one unit increases the odds for the class `virginica` the most? Explain your choice.

**Solution:**

I guess that increasing features $3$ or $4$ will increase the chances for the class 'virgincia' the most. Because:

1. Distribution of one feature: If we look at the images on the diagonal, we notice that the distributions of the individual input features are different. But in the case of the first two, large overlaps are already visible and a shift of the blue (not 'virgincia') by one to the right (corresponding to an increase of the input feature by 1) would further increase them. But if input features 3 or 4 are increased by one each, there are much more new overlaps, i.e. the difference from before to after is much larger than for the first two features.

2. Distribution of a pair of features: Clearly, with an increase of 1, we want the considered data point to be moved into the cluster of data points for the 'virginica' class. For example, if we increase the feature 'sepal width', we move away from both distributions, i.e. it would not increase our chances for the class 'virginica'. The most overlap would be for features 3 and 4, which is why those are my guess.

In [ ]:
# Plot the dataframe
fig = sb.pairplot(data = iris_df, hue = 'virginica', height = 3.5)

**iii)** Fit a regression model with the function `sklearn.linear_model.LogisticRegression` and extract the estimated model parameters $\beta$ from it. Now we want to determine 
the odds ratio of the parameters. Compared to the simple example from part a), 
a logistic model provides a probability conditional on the input data $x$, i.e. 
the odds is given by
$$
\text{odds}_\text{logreg} = \frac{\mathbb{P}(Y = 1 \mid x)}{1- \mathbb{P}(Y = 1 \mid x)}.
$$
Therefore instead of using another dice, the stochastic model is now modified by increasing a component of $x$. Denote with $x_{x_i \to x_i +1}$ a vector where the $i$-th component (feature) is increased by one unit. Thus, the following representation of the odds ratio for the i-th parameter results from the lecture:
$$
\text{odds ratio}_i = \frac{\frac{\mathbb{P}(Y = 1 \mid x_{x_i \to x_i +1})}{1- \mathbb{P}(Y = 1 \mid x_{x_i \to x_i +1})}}{\frac{\mathbb{P}(Y = 1 \mid x)}{1- \mathbb{P}(Y = 1 \mid x)}} = \exp(\beta_i).
$$
What are the odds ratios of your fitted logistic regression model, and which 
feature increases the odds for the class 'virginica' the most? How does your 
choice fit with your thoughts from part II)?

**Solution:**

In [ ]:
from sklearn.linear_model import LogisticRegression

# Get the input and outcome of your iris dataframe and move it to numpy
X = iris_df.iloc[:, 0:-1].to_numpy()
Y = iris_df.iloc[:, -1].to_numpy()

# Fit a logistic regression model on these data
model_logreg = LogisticRegression().fit(X, Y)

# Plot the result in a clear table
beta = np.append(model_logreg.intercept_, model_logreg.coef_)
pd.DataFrame(data = {"label": ["b0", "b1", "b2", "b3", "b4"], "beta": beta, "Odds ratio": np.round(np.exp(beta), 2)})

 # 3) Prediction of Purchase Prices of Houses
Load the dataset `sheet_2_house_price_data.csv`. This dataset contains the purchase prices of $10.000$ houses in Germany and includes the `size` (in m²), `location` ($0=$ bad, $1=$ good, $2=$ very good and $3=$ luxury) and year of construction (`year`).

**a)** Fit a linear regression model and calculate the mean squared error (MSE) of the dataset.

In [ ]:
from sklearn.linear_model import LinearRegression

# Prepare dataset for training
house_data = pd.read_csv("sheet_2_house_price_data.csv")
X = house_data.iloc[:, 0:-1].to_numpy()
Y = house_data.iloc[:, -1].to_numpy()

# Fit a linear regression model
linear_model = LinearRegression().fit(X,Y)

# Calculate the MSE
'MSE:  ' + str(np.round(np.mean((linear_model.predict(X) - Y)**2), 2))

**b)** How can this dataset be examined to see whether feature interactions affect the output and, if so, which one? A visual explanation is enough.

In [ ]:
## Test for interactions 'size' ~ 'location'
fig = sb.FacetGrid(house_data, col="location", height = 6)
fig.map_dataframe(sb.scatterplot, x="size", y="price")
fig.add_legend()
fig.set_xlabels("size")
fig = fig.set_ylabels("price")

# Seems to be interacting features

Assuming they would not interact, a change in 'location' should only provide a uniform shift with respect to the y-axis, since this would be a constant in the linear model. But the slope changes when the location is changed, so it must be an interaction of 'size' ~ ' location'.

In [ ]:
## Test for interactions 'size' ~ 'year'
plt.figure(figsize=(15, 10))
fig = sb.scatterplot(data = house_data, x="size", y="price", hue = "year")

# Seems to be interacting features

Assuming they would not interact, a change in 'year' should only provide a uniform shift with respect to the y-axis, since this would be a constant in the linear model. But the slope changes when the year is changed, so it must be an interaction of 'size' ~ ' year'.

In [ ]:
## Test for interactions 'year' ~ 'location'
fig = sb.FacetGrid(house_data, col="location", height = 6)
fig.map_dataframe(sb.scatterplot, x="year", y="price")
fig.add_legend()
fig.set_xlabels("year")
fig = fig.set_ylabels("price")

# Seems to be non-interacting features

All plots look the same (except for a shift in height), so no interaction (at least not a strong one) is apparent. 

**c)** Fit a linear regression model on these data by manually adding the relevant interactions. Then calculate the MSE again.

In [ ]:
house_data_interact = house_data
house_data_interact['size_year'] = house_data_interact['size'] * house_data_interact['year']
house_data_interact['size_loc'] = house_data_interact['size'] * house_data_interact['location']

X = house_data_interact.iloc[:, [0,1,2,4,5]].to_numpy()
Y = house_data_interact.iloc[:, [3]].to_numpy()

# Fit a linear regression model
linear_model = LinearRegression().fit(X,Y)

# Calculate the MSE
'MSE:  ' + str(np.round(np.mean((linear_model.predict(X) - Y)**2), 2))

## 4) Advanced: Logistic Regression
Similar to linear regression, logistic regression assumes that there are no interactions between the features, even though this is rarely the case in reality. This also means that the odds ratio is constant for the estimated parameters $\beta$ independent of the input data $x$, although the logistic model outputs a probability conditional on the data ($\mathbb{P}(Y = 1 \mid x)$).
Let's consider the following logistic model with an interaction term and numerical features
$$
\mathbb{P}(Y = 1 \mid x) = f(x) = \text{logistic}\left(\beta_0 + x_1 \beta_1 + x_2 \beta_2 + x_3 \beta_3 + \beta_{12} x_1 x_2 + \beta_{33} x_3^2\right).
$$

Calculate the odds ratio for each parameter $\beta_1, \beta_2$ and $\beta_3$ if we
increase the respective feature value by one unit.

**Solution:**

It holds for $x$
$$
\begin{align}
\text{odds} &= \frac{\mathbb{P}(Y = 1 \mid x)}{ 1 - \mathbb{P}(Y=1 \mid x)} = \frac{f(x)}{1-f(x)} = \frac{\frac{1 }{1 + \exp\left(- \left( \beta_0 + x_1 \beta_1 + x_2 \beta_2 + x_3 \beta_3 + \beta_{12} x_1 x_2 + \beta_{33} x_3^2\right) \right)}}{1 - \frac{1 }{1 + \exp\left(- \left( \beta_0 + x_1 \beta_1 + x_2 \beta_2 + x_3 \beta_3 + \beta_{12} x_1 x_2 + \beta_{33} x_3^2\right) \right)}} \\
& = \frac{1 }{1 + \exp\left(- \left( \beta_0 + x_1 \beta_1 + x_2 \beta_2 + x_3 \beta_3 + \beta_{12} x_1 x_2 + \beta_{33} x_3^2\right) \right) -1} \\
&= \exp\left( \beta_0 + x_1 \beta_1 + x_2 \beta_2 + x_3 \beta_3 + \beta_{12} x_1 x_2 + \beta_{33} x_3^2\right) 
\end{align}.
$$

Hence, we get
$$
\text{odds ratio}_{X_1} = \frac{\exp\left( \beta_0 + (x_1 +1) \beta_1 + x_2 \beta_2 + x_3 \beta_3 + \beta_{12} (x_1 + 1) x_2 + \beta_{33} x_3^2\right)}{\exp\left( \beta_0 + x_1 \beta_1 + x_2 \beta_2 + x_3 \beta_3 + \beta_{12} x_1 x_2 + \beta_{33} x_3^2\right)} = \exp\left(\beta_1 + \beta_{12} x_2\right)
$$
$$
\text{odds ratio}_{X_2} = \frac{\exp\left( \beta_0 + x_1 \beta_1 + (x_2 + 1) \beta_2 + x_3 \beta_3 + \beta_{12} x_1 (x_2 + 1) + \beta_{33} x_3^2\right)}{\exp\left( \beta_0 + x_1 \beta_1 + x_2 \beta_2 + x_3 \beta_3 + \beta_{12} x_1 x_2 + \beta_{33} x_3^2\right)} = \exp\left(\beta_2 + \beta_{12} x_1\right)
$$
$$
\text{odds ratio}_{X_3} = \frac{\exp\left( \beta_0 + x_1 \beta_1 + x_2 \beta_2 + (x_3 + 1) \beta_3 + \beta_{12} x_1 x_2 + \beta_{33} (x_3 +1)^2\right)}{\exp\left( \beta_0 + x_1 \beta_1 + x_2 \beta_2 + x_3 \beta_3 + \beta_{12} x_1 x_2 + \beta_{33} x_3^2\right)} = \exp\left(2\beta_3 + 2\beta_{33} x_3\right)
$$